# Examining Point Clouds

This notebook is for examining the point cloud data generated by the first analysis phase (phase pointcloud). You do not need to have run Spyral to run this notebook, you simply must have created a valid Spyral input configuration. Plots of the individual traces point clouds in two and three dimensions can be made to check the status of the results. This is helpful for testing the parameters generated by the first phase. It actually runs a mini-version of phase 1. Note that the data generated here is NOT saved. This is only for testing.

### Imports

First we import all of the necessary modules

In [ ]:
from spyral.core.point_cloud import PointCloud
from spyral.core.run_stacks import form_run_string
from spyral.trace.get_event import GetEvent, GET_DATA_TRACE_START, GET_DATA_TRACE_STOP
from spyral.trace.get_legacy_event import GetLegacyEvent
from spyral.phases.pointcloud_phase import get_event_range
from spyral.correction import create_electron_corrector
from spyral.core.pad_map import PadMap

from spyral import PadParameters, GetParameters, FribParameters, DetectorParameters, INVALID_PATH, PointcloudPhase, PointcloudLegacyPhase

import h5py as h5
import numpy.random as random
import numpy as np
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def find_trace_from_padid(event: GetEvent | GetLegacyEvent, pad_id: int) -> int:
    for idx, trace in enumerate(event.traces):
        if trace.hw_id.pad_id == pad_id:
            return idx
    return -1

### Configuration

Now we'll load the configuration we want to use. Configurations are stored in JSON files using the conventions defined by the example file shipped with the repository (config.json). By default, this notebook reads the example file, but this can be changed. Additionally, once the config is loaded, you can always tweak the fields for rapid testing of different parameters.

We then hand off the workspace configuration to the Workspace class. The Workspace helps us handle paths to various files.

In [ ]:
# Some parameters
trace_path = Path("/path/to/raw/attpc/traces/")
workspace_path = Path("/path/to/your/workspace/")

# Are we analyzing legacy data?
is_legacy: bool = False

# Pad mapping. We use defaults here
pad_params = PadParameters(
    is_default=True,
    is_default_legacy=False,
    pad_geometry_path=INVALID_PATH,
    pad_gain_path=INVALID_PATH,
    pad_time_path=INVALID_PATH,
    pad_electronics_path=INVALID_PATH,
    pad_scale_path=INVALID_PATH,
)

# AT-TPC GET trace analysis
get_params = GetParameters(
    baseline_window_scale=20.0,
    peak_separation=50.0,
    peak_prominence=20.0,
    peak_max_width=50.0,
    peak_threshold=40.0,
)

# AT-TPC FRIBDAQ trace analysis (or legacy GET extension)
frib_params = FribParameters(
    baseline_window_scale=100.0,
    peak_separation=50.0,
    peak_prominence=20.0,
    peak_max_width=500.0,
    peak_threshold=100.0,
    ic_delay_time_bucket=1100,
    ic_multiplicity=1,
    correct_ic_time=True,
)

# Detector properties
det_params = DetectorParameters(
    magnetic_field=2.85,
    electric_field=45000.0,
    detector_length=1000.0,
    beam_region_radius=25.0,
    micromegas_time_bucket=10.0,
    window_time_bucket=560.0,
    get_frequency=6.25,
    garfield_file_path=Path("/path/to/some/garfield.txt"),
    do_garfield_correction=False,
)

### Data Loading

Now that our configuration is loaded, we can start reading and analyzing some data. Step one is to access the raw trace datafile. This means that you need to pick a run to analyze; we store the run number in a variable for later reference. To analyze a different run simply change the run number.

We then ask the workspace to retrieve the trace file path for our selected run number. We then use the h5py library to open the associated h5 file. 

In [ ]:
run_number = 16 # pick a run
trace_file_path = trace_path / f"{form_run_string(run_number)}.h5"
trace_file = h5.File(trace_file_path, "r")

Our file is now loaded. Now we need to navigate to the correct group of the h5 structure. The traces are stored in the 'get' group.

In [ ]:
trace_group: h5.Group = trace_file['get']

With a group loaded, we can now access the actual trace data! The default loading behavior is to select a random event (set of traces), since when testing you'll want to test your parameters on many different events. However, one can also fix the event by setting the event number to a constant value for debugging. Once the event number is chosen we then retrieve the raw trace data from the h5 file as a Dataset. Note that this operation can fail sometimes, if an event was omitted for some reason. If that happens just select a different event number.

In [ ]:
# Ask the trace file for the range of events
min_event, max_event = get_event_range(trace_file)
rng = random.default_rng()
# Select a random event
event_number = rng.integers(min_event, max_event)
# Can always overwrite with hardcoded event number if needed
# event_number = 2941
print(f'Event: {event_number}')

event_data: h5.Dataset = trace_group[f'evt{event_number}_data']
event = None
correction_path: Path
# Load either a legacy or contemporary GET daq event, and create our assets
if is_legacy:
    event = GetLegacyEvent(event_data, event_number, get_params, frib_params, rng)
    phase = PointcloudLegacyPhase(get_params, frib_params, det_params, pad_params)
    phase.create_assets(workspace_path)
    correction_path = phase.electron_correction_path
else:
    event = GetEvent(event_data, event_number, get_params, rng)
    phase = PointcloudPhase(get_params, frib_params, det_params, pad_params)
    phase.create_assets(workspace_path)
    correction_path = phase.electron_correction_path

pad_map = PadMap(pad_params)


### Analyzing

Now that we've loaded our data, it's time to do some analysis! Step one is just to see what some traces look like, without any analysis at all. Again here, we're going to pick a random trace to look at, but you may want to pick a specific trace depending on your needs.

In [ ]:
trace_number = random.randint(0, len(event_data))
# trace_number = find_trace_from_padid(event, 397)
raw_trace_data = event_data[trace_number]
time_bucket_range = np.arange(start=0, stop=512)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=raw_trace_data[GET_DATA_TRACE_START:GET_DATA_TRACE_STOP], mode="lines", name=f"Raw Trace {trace_number}")
)
fig.add_trace(
    go.Scatter(x=time_bucket_range, y=event.traces[trace_number].trace, mode="lines",name=f"Baseline Corrected Trace {trace_number}")
)
print(f"Trace Number: {trace_number}")
print(f"Trace Hardware: {event.traces[trace_number].hw_id}")
peak_amps = []
peak_cents = []
peak_left = []
peak_left_amps = []
peak_right = []
peak_right_amps = []
for peak in event.traces[trace_number].get_peaks():
    peak_amps.append(peak.amplitude)
    peak_cents.append(np.floor(peak.centroid))
    peak_left.append(peak.positive_inflection)
    peak_right.append(peak.negative_inflection)
    peak_left_amps.append(event.traces[trace_number].trace[int(peak.positive_inflection)])
    peak_right_amps.append(event.traces[trace_number].trace[int(peak.negative_inflection)])
print(f"Peak centroids: {peak_cents}")
fig.add_trace(
    go.Scatter(x=peak_cents, y=peak_amps, mode="markers", name="Peaks")
)
fig.add_trace(
    go.Scatter(x=peak_left, y=peak_left_amps, mode="markers", name="Peak Left Edges")
)
fig.add_trace(
    go.Scatter(x=peak_right, y=peak_right_amps, mode="markers", name="Peak Right Edges")
)
fig.update_legends()
fig.update_layout(
    xaxis_title="Time Bucket",
    yaxis_title="Amplitude",
    showlegend=True
)
fig.show()

Above you should see the plot of the raw trace as well as the baseline corrected trace. The baseline corrected trace is computed using a low-pass filter. This is done by passing the data to the GetEvent class which performs some signal analysis, identifying the peaks in the signal. The peaks are labeled with their centroids and left and right edges. To look at different traces, you can run the above cell over and over again; it will select a random trace in the event each time.

Now that peaks have been found, we can form a point cloud! Here we'll also apply the eletric field correction from Garfield. If you haven't run Spyral at all, this cell might take a bit to run, as the electric field correction will need to be generated if it is turned on. 

In [ ]:
cloud = PointCloud()

# Do the electric field correction if requested
corrector = None
if correction_path.exists():
    corrector = create_electron_corrector(correction_path)

cloud.load_cloud_from_get_event(event, pad_map)
hover_text = [f"Pad ID: {int(point[5])}" for point in cloud.cloud] # We'll use this later

fig = make_subplots(2, 1, row_heights=[0.66, 0.33], specs=[[{"type": "xy"}], [{"type": "scene"}]])
fig.add_trace(
    go.Scatter3d(
        x=cloud.cloud[:, 2], 
        y=cloud.cloud[:, 0], 
        z=cloud.cloud[:, 1], 
        mode="markers",
        text = hover_text,
        hovertemplate="X: %{y:.2f}<br>Y: %{z:.2f}<br>Z: %{x:.2f}<br>%{text}",
        marker= {
            "size": 3, 
            "color": cloud.cloud[:, 3], 
            "showscale": True
            }, 
        name="Point Cloud"
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=cloud.cloud[:, 0], 
        y=cloud.cloud[:, 1], 
        mode="markers",
        text = hover_text,
        hovertemplate="X: %{x:.2f}<br>Y: %{y:.2f}<br>%{text}",
        marker= {
            "color": cloud.cloud[:, 3], 
            "showscale": True
        }, 
        name="XY Projection"),
    row=1,
    col=1
)
fig.update_layout(
    xaxis_title = "X (mm)",
    yaxis_title = "Y (mm)",
    xaxis_range=[-300.0, 300.0],
    yaxis_range=[-300.0, 300.0],
    scene = {
        "xaxis_title": "Z (Time Buckets)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        },
        "xaxis_range": [0.0, 512.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
    },
    width = 1000,
    height = 1500,
    showlegend=False
)
fig.show()

Above, you should see your point cloud plotted in 3D as well as the X-Y plane (pad plane) projection. The marker color indicates the integral of the peak used to make the point in the point cloud. If you hover over one of the points in either plot, you'll see a label which shows the coordinate position as well as the trace and peak number which produced the point. This can be used to pick specific traces to examine!

Notice that the z-axis is still in Time Buckets. We would like to convert this time axis into a position. To do this we use the reference time of the window and micromegas mesh (i.e. the position of the ends of the detector within the trigger). These values have to be estimated from the data. Typically this is handled by looking for window events (events where the beam reacted with the window), because they typically span the entire volume of the detector. Once you've set these values in your config, run the cell below to re-plot the point cloud with calibrated z-position.

In [ ]:

cloud.calibrate_z_position(det_params.micromegas_time_bucket, det_params.window_time_bucket, det_params.detector_length, efield_correction=corrector)

fig = go.Figure()
fig.add_trace(
    go.Scatter3d(
        x=cloud.cloud[:, 2], 
        y=cloud.cloud[:, 0], 
        z=cloud.cloud[:, 1], 
        mode="markers", 
        marker= {
            "size": 3, 
            "color": cloud.cloud[:, 4], 
            "showscale": True
        }, 
        name="Point Cloud"
    )
)
fig.update_layout(
    scene = {
        "xaxis_range": [0.0, 1000.0],
        "yaxis_range": [-300.0, 300.0],
        "zaxis_range": [-300.0, 300.0],
        "xaxis_title": "Z (mm)",
        "yaxis_title": "X (mm)",
        "zaxis_title": "Y (mm)",
        "aspectratio": {
            "x": 3.3,
            "y": 1.0,
            "z": 1.0
        }
    },
    height=750,
)
fig.show()

Everything should look more or less the same. You've now generated a point cloud! To examine a new event, you can simply re-run the entire notebook, which will select a new random event from the run of interest.

There is still more you can look at however. You can even plot some interesting physics! Below is an example intended to try and plot a Bragg curve from the point cloud.

In [ ]:
# Plot r-Charge projection
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=np.linalg.norm(cloud.cloud[:, :3], axis=1), y=cloud.cloud[:, 4], mode="markers", marker={"size": 5})
)
fig.update_layout(
    xaxis_title="Position (mm)",
    yaxis_title="Integral"
)
fig.show()

### Conclusion

That is a basic analysis of the traces and the point cloud data! With well tuned parameters, you're now ready to run the phase 1 analysis. Follow the instructions in the README to do this. Once thats done, you can move on to the next stage, generating and identifying clusters within the point clouds.